<a href="https://colab.research.google.com/github/tfranke0814/pytorch-deep-learning/blob/main/02_pytorch_classification_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

08:40:32 - https://www.youtube.com/watch?v=LyJtbe__2i0&t=8h40m32s

In [ ]:
import torch
import matplotlib as plt